<a href="https://colab.research.google.com/github/hamsika04/5A_AILAB/blob/main/1BM22CS054_Week4_A__MisplaceTiles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
class Node:
    def __init__(self, data, level, fval):
        self.data = data
        self.level = level
        self.fval = fval

    def generate_child(self):
        """ Generate child nodes by moving the blank space (up, down, left, right). """
        x, y = self.find_blank()
        moves = {
            "left": (x, y-1),
            "right": (x, y+1),
            "up": (x-1, y),
            "down": (x+1, y)
        }
        children = []
        for direction, (new_x, new_y) in moves.items():
            child_data = self.move_blank(x, y, new_x, new_y)
            if child_data:
                child_node = Node(child_data, self.level + 1, 0)
                children.append((child_node, direction))  # Store child node along with direction
        return children

    def move_blank(self, x1, y1, x2, y2):
        """ Move the blank space to a new position if within bounds. """
        if 0 <= x2 < len(self.data) and 0 <= y2 < len(self.data[0]):
            new_data = [row[:] for row in self.data]  # Copy the puzzle
            new_data[x1][y1], new_data[x2][y2] = new_data[x2][y2], new_data[x1][y1]  # Swap blank
            return new_data
        return None

    def find_blank(self):
        """ Find the position of the blank space ('_'). """
        for i, row in enumerate(self.data):
            if '_' in row:
                return i, row.index('_')
        raise ValueError("Blank space ('_') not found in the puzzle state.")

class Puzzle:
    def __init__(self, size):
        self.size = size
        self.open = []
        self.closed = []

    def get_input(self):
        """ Accept the puzzle state as input from the user. """
        return [input().split() for _ in range(self.size)]

    def f(self, start, goal):
        """ Calculate f(x) = g(x) + h(x), where g(x) is the level and h(x) is the heuristic. """
        return start.level + self.misplaced_tiles_heuristic(start.data, goal)

    def misplaced_tiles_heuristic(self, start_data, goal):
        """ Calculate the number of misplaced tiles heuristic for the current puzzle state. """
        misplaced_count = 0
        for i in range(self.size):
            for j in range(self.size):
                if start_data[i][j] != '_' and start_data[i][j] != goal[i][j]:
                    misplaced_count += 1  # Count the number of tiles not in their goal position
        return misplaced_count

    def find_position(self, state, value):
        """ Find the position of a specific value in the puzzle. """
        for i in range(self.size):
            for j in range(self.size):
                if state[i][j] == value:
                    return i, j

    def process(self):
        """ Main method to solve the puzzle using A* search. """
        print("Enter the start state matrix:")
        start_data = self.get_input()
        print("Enter the goal state matrix:")
        goal = self.get_input()

        start_node = Node(start_data, 0, 0)
        start_node.fval = self.f(start_node, goal)
        self.open.append(start_node)

        while self.open:
            current_node = self.open.pop(0)
            self.display_state(current_node.data)

            if self.misplaced_tiles_heuristic(current_node.data, goal) == 0:
                print("\nGoal reached!")
                break

            children = current_node.generate_child()
            move_details = []

            for child, direction in children:
                child.fval = self.f(child, goal)
                move_details.append((child.data, child.level, child.fval, direction))
                self.open.append(child)

            print("\nPossible moves (child states with g(n), h(n), f(n)): ")
            for data, level, fval, direction in move_details:
                h_value = self.misplaced_tiles_heuristic(data, goal)
                print(f"\nMove: {direction.capitalize()}")
                print(f"State:\n{self.format_state(data)}")
                print(f"g(n): {level}, h(n): {h_value}, f(n): {fval}")

            self.closed.append(current_node)
            self.open.sort(key=lambda node: node.fval)  # Sort by f-value (least-cost first)

    def display_state(self, data):
        """ Display the current state of the puzzle. """
        print("\nCurrent state:")
        print(self.format_state(data))

    def format_state(self, data):
        """ Format the state for display. """
        return "\n".join(" | ".join(row) for row in data)

# Run the puzzle solver
if __name__ == "__main__":
    puz = Puzzle(3)
    puz.process()


Enter the start state matrix:
2 8 3
1 6 4
7 _ 5
Enter the goal state matrix:
1 2 3
8  _ 4
7 6 5

Current state:
2 | 8 | 3
1 | 6 | 4
7 | _ | 5

Possible moves (child states with g(n), h(n), f(n)): 

Move: Left
State:
2 | 8 | 3
1 | 6 | 4
_ | 7 | 5
g(n): 1, h(n): 5, f(n): 6

Move: Right
State:
2 | 8 | 3
1 | 6 | 4
7 | 5 | _
g(n): 1, h(n): 5, f(n): 6

Move: Up
State:
2 | 8 | 3
1 | _ | 4
7 | 6 | 5
g(n): 1, h(n): 3, f(n): 4

Current state:
2 | 8 | 3
1 | _ | 4
7 | 6 | 5

Possible moves (child states with g(n), h(n), f(n)): 

Move: Left
State:
2 | 8 | 3
_ | 1 | 4
7 | 6 | 5
g(n): 2, h(n): 3, f(n): 5

Move: Right
State:
2 | 8 | 3
1 | 4 | _
7 | 6 | 5
g(n): 2, h(n): 4, f(n): 6

Move: Up
State:
2 | _ | 3
1 | 8 | 4
7 | 6 | 5
g(n): 2, h(n): 3, f(n): 5

Move: Down
State:
2 | 8 | 3
1 | 6 | 4
7 | _ | 5
g(n): 2, h(n): 4, f(n): 6

Current state:
2 | 8 | 3
_ | 1 | 4
7 | 6 | 5

Possible moves (child states with g(n), h(n), f(n)): 

Move: Right
State:
2 | 8 | 3
1 | _ | 4
7 | 6 | 5
g(n): 3, h(n): 3, f(n): 6

M